In [1]:
import torch
import torch_geometric.transforms as T
import matplotlib.pyplot as plt
import collections
import numpy as np
import pandas as pd
import networkx as nx
from torch_geometric.data import Data
from torch_geometric.utils import to_networkx
import scipy
from scipy.io import loadmat as lm

from torch_geometric.utils.convert import  from_scipy_sparse_matrix

In [5]:
data_ch = torch.load("./data.pt")[0]

In [15]:
collections.Counter(data_ch.y.tolist())

Counter({0: 456, 3: 521, 4: 387, 2: 453, 1: 460})

In [42]:
y_true = data_Photo.y.tolist()

In [43]:
y_pred=list(np.random.randint(0, 5, len(y_true)))

In [44]:
from sklearn.metrics import f1_score

print(f1_score(y_pred,y_true,average='micro'))
f1_score(y_pred,y_true,average='macro')

0.11751633986928105


0.08764362531811375

In [32]:
from sklearn.metrics import f1_score

print(f1_score(y_true,y_pred,average='micro'))
f1_score(y_true,y_pred,average='macro')

0.18972332015810275


0.18908907548114942

**ПРОБЛЕМЫ ВОПРОСЫ**:
- BlogCatalog, CoCit, Wiki имебют матрицу groupб утвреждается что это лейблы но как быть если у BlogCatalog, Wiki на каждую вершину по более чем одному лейблу -  поставила всем как - фичи
- Epinion, Arxiv не имеют ни лейблов ни фичей

In [3]:
def to_networkx(data, node_attrs=None, edge_attrs=None):
    r"""Converts a data object graph to a networkx graph.

    Args:
        data (torch_geometric.data.Data): The data object.
        node_attrs (iterable of str, optional): The node attributes to be
            copied. (default: :obj:`None`)
        edge_attrs (iterable of str, optional): The edge attributes to be
            copied. (default: :obj:`None`)
    """

    G = nx.Graph()
    if node_attrs is not None:
        
        for i,attr in enumerate(node_attrs):
            G.add_node(i, feature = str(attr.tolist()),label = int(data.y[i]))
    else:
        G.add_nodes_from(list(range((data.num_nodes))))
    
    for i, (u, v) in enumerate(data.edge_index.t().tolist()):
        G.add_edge(u, v)
    
    return G


In [60]:
#тут мульти-лейбл
Blogcatalog = lm('datasets/blogcatalog.mat')
blogcatalog_edgeindex,_=from_scipy_sparse_matrix(Blogcatalog['network'])
#blogcatalog_x = torch.tensor(scipy.sparse.csr_matrix.toarray(Blogcatalog['group']))
data_BlogCatalog = Data(edge_index = blogcatalog_edgeindex)
G_BlogCatalog = to_networkx(data_BlogCatalog)
#nx.from_scipy_\sparse_matrix(data_Blogcatalog['network'])

In [61]:
CoCit = lm('datasets/CoCit.mat')
CoCit_edgeindex,_=from_scipy_sparse_matrix(CoCit['network'])
CoCit_y_matrix = torch.tensor(scipy.sparse.csr_matrix.toarray(CoCit['group']))
CoCit_y = CoCit_y_matrix.nonzero().t()[1]
data_CoCit = Data(edge_index = CoCit_edgeindex, y = CoCit_y)
G_CoCit=to_networkx(data_CoCit)

c:\users\user\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:4: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  ..\torch\csrc\utils\python_arg_parser.cpp:882.)
  after removing the cwd from sys.path.


In [ ]:
#тут мульти-лейбл
Wiki = lm('datasets/Wiki.mat')
Wiki_edgeindex,_=from_scipy_sparse_matrix(Wiki['network'])
#Wiki_x = torch.tensor(scipy.sparse.csr_matrix.toarray(Wiki['group']))
data_Wiki = Data(edge_index = Wiki_edgeindex)
G_Wiki=to_networkx(data_Wiki)

In [ ]:
#провекра на симметричность а значит на ненаправленность
np.all(scipy.sparse.csr_matrix.toarray(Blogcatalog['network']).transpose() == scipy.sparse.csr_matrix.toarray(Blogcatalog['network']))

In [ ]:
#провекра на симметричность а значит на ненаправленность
np.all(scipy.sparse.csr_matrix.toarray(CoCit['network']).transpose() == scipy.sparse.csr_matrix.toarray(CoCit['network']))

In [ ]:
#провекра на симметричность а значит на ненаправленность
np.all(scipy.sparse.csr_matrix.toarray(Wiki['network']).transpose() == scipy.sparse.csr_matrix.toarray(Wiki['network']))

In [ ]:
with open('datasets/soc-Epinions1.txt','r') as f:
    data_Epinion = f.readlines()

new_list=list(map(lambda x: x.split('\t'),data_Epinion[4:]))
#print(new_list)
dataedge_ep = list(map(lambda x: [int(x[0]),int(x[1].split('\n')[0])],new_list))
set1=set(collections.Counter((np.array(dataedge_ep)).transpose()[1]).keys()) | set(collections.Counter((np.array(dataedge_ep)).transpose()[0]).keys())
map_ep={}
for j,i in enumerate(set1):
    map_ep[i] = j
dataedge_ep_new = list(map(lambda x: [map_ep[x[0]],map_ep[x[1]]], dataedge_ep))
data_edge_index_Epinion = torch.tensor(dataedge_ep_new).t()
data_Epinion_torch = Data(edge_index=data_edge_index_Epinion)
G_Epinion = to_networkx(data_Epinion_torch)

In [3]:
with open('datasets/CA-GrQc.txt','r') as f:
    data_Arxiv = f.readlines()

new_list=list(map(lambda x: x.split('\t'),data_Arxiv[4:]))
#print(new_list)
dataedge = list(map(lambda x: [int(x[0]),int(x[1].split('\n')[0])],new_list))
set2=set(collections.Counter((np.array(dataedge)).transpose()[1]).keys()) | set(collections.Counter((np.array(dataedge)).transpose()[0]).keys())
map_ar={}
for j,i in enumerate(set2):
    map_ar[i] = j
    
dataedge_ar_new = list(map(lambda x: [map_ar[x[0]],map_ar[x[1]]], dataedge))

data_edge_index_Arxiv = torch.tensor(dataedge_ar_new).t()
data_Arxiv_torch = Data(edge_index=data_edge_index_Arxiv)
G_Arxiv = to_networkx(data_Arxiv_torch)

In [33]:
from torch_geometric.datasets import Planetoid

data_Cora =Planetoid(root='/tmp/Cora', name='Cora',transform=T.NormalizeFeatures())[0]
#data_cora =dataset[0]
data_Citeseer =Planetoid(root='/tmp/Citeseer', name='Citeseer',transform=T.NormalizeFeatures())[0]
data_Pubmed =Planetoid(root='/tmp/Pubmed', name='Pubmed',transform=T.NormalizeFeatures())[0]

from torch_geometric.datasets import Amazon

data_Photo =Amazon(root='/tmp/Photo', name='Photo',transform=T.NormalizeFeatures())[0]
data_Computers =Amazon(root='/tmp/Computers', name='Computers',transform=T.NormalizeFeatures())[0]

from torch_geometric.datasets import Coauthor

data_CS =Coauthor(root='/tmp/CS', name='CS',transform=T.NormalizeFeatures())[0]
data_Physics =Coauthor(root='/tmp/Physics', name='Physics',transform=T.NormalizeFeatures())[0]

from torch_geometric.datasets import Reddit


#social networks: Reddit, Flickr
from torch_geometric.datasets import Flickr
data_Flickr =Flickr(root='/tmp/Flickr',transform=T.NormalizeFeatures())[0]
# data_Reddit = Reddit(root='/tmp/Reddit',transform=T.NormalizeFeatures()) очень тяжелый 


from torch_geometric.datasets import PPI

data_PPI = PPI(root='/tmp/PPI', split="train",transform=T.NormalizeFeatures())[0] #тут только train

In [4]:
datasets = [data_Cora, data_Citeseer, data_Pubmed, data_Photo, data_Computers, data_CS, data_Physics,data_Flickr,data_CoCit,data_BlogCatalog,data_Wiki,data_Arxiv_torch,data_Epinion_torch]
datasets_names = ['Cora', 'Citeseer', 'Pubmed', 'Photo', 'Computers', 'CS', 'Physics','Flickr','CoCit','BlogCatalog','Wikipedia','Arxiv','Epinion']

NameError: name 'data_CoCit' is not defined

In [63]:
graphs = []
for dataset in datasets:
    graphs.append(to_networkx(dataset,dataset.x))

In [ ]:
datasets =[data_Cora, data_Citeseer,  data_Photo, data_Arxiv_torch]
datasets_names = ['Cora','Citeseer','Photo','Arxiv']

In [64]:
cora_graph = to_networkx(data_Cora,data_Cora.x)

In [6]:
chameleon_graph = to_networkx(data_ch,data_ch.x)

In [8]:
import math
one = 0
two = 0
three = 0
mean_1=0
mean_2=0
for edge in chameleon_graph.edges():
    mean_1 +=  chameleon_graph.nodes('label')[edge[0]]
    mean_2 += chameleon_graph.nodes('label')[edge[1]]
mean_1 = mean_1/len( chameleon_graph.edges())
mean_2 = mean_2/len( chameleon_graph.edges())

for edge in chameleon_graph.edges():
    one+= (chameleon_graph.nodes('label')[edge[0]]-mean_1)*(chameleon_graph.nodes('label')[edge[1]]-mean_2)
    two += (chameleon_graph.nodes('label')[edge[0]]-mean_1)*(chameleon_graph.nodes('label')[edge[0]]-mean_1)
    three += (chameleon_graph.nodes('label')[edge[1]]-mean_2)*(chameleon_graph.nodes('label')[edge[1]]-mean_2)

d = math.sqrt(two*three)
print(one/d)

0.27463230751013357


In [99]:
citeseer_graph = to_networkx(data_Citeseer,data_Citeseer.x)
import math
one = 0
two = 0
three = 0
mean_1=0
mean_2=0
for edge in citeseer_graph.edges():
    mean_1 +=  citeseer_graph.nodes('label')[edge[0]]
    mean_2 += citeseer_graph.nodes('label')[edge[1]]
mean_1 = mean_1/len( citeseer_graph.edges())
mean_2 = mean_2/len( citeseer_graph.edges())

for edge in citeseer_graph.edges():
    one+= (citeseer_graph.nodes('label')[edge[0]]-mean_1)*(citeseer_graph.nodes('label')[edge[1]]-mean_2)
    two += (citeseer_graph.nodes('label')[edge[0]]-mean_1)*(citeseer_graph.nodes('label')[edge[0]]-mean_1)
    three += (citeseer_graph.nodes('label')[edge[1]]-mean_2)*(citeseer_graph.nodes('label')[edge[1]]-mean_2)

d = math.sqrt(two*three)
print(one/d)

0.6840364066412794


In [101]:
photo_graph = to_networkx(data_Photo,data_Photo.x)
one = 0
two = 0
three = 0
mean_1=0
mean_2=0
for edge in photo_graph.edges():
    mean_1 +=  photo_graph.nodes('label')[edge[0]]
    mean_2 += photo_graph.nodes('label')[edge[1]]
mean_1 = mean_1/len( photo_graph.edges())
mean_2 = mean_2/len( photo_graph.edges())

for edge in photo_graph.edges():
    one+= (photo_graph.nodes('label')[edge[0]]-mean_1)*(photo_graph.nodes('label')[edge[1]]-mean_2)
    two += (photo_graph.nodes('label')[edge[0]]-mean_1)*(photo_graph.nodes('label')[edge[0]]-mean_1)
    three += (photo_graph.nodes('label')[edge[1]]-mean_2)*(photo_graph.nodes('label')[edge[1]]-mean_2)

d = math.sqrt(two*three)
print(one/d)

0.8539507520546509


In [ ]:
import math
one = 0
two = 0
three = 0
mean_1=0
mean_2=0
for edge in cora_graph.edges():
    mean_1 +=  cora_graph.nodes('label')[edge[0]]
    mean_2 += cora_graph.nodes('label')[edge[1]]
mean_1 = mean_1/len( cora_graph.edges())
mean_2 = mean_2/len( cora_graph.edges())

for edge in cora_graph.edges():
    one+= (cora_graph.nodes('label')[edge[0]]-mean_1)*(cora_graph.nodes('label')[edge[1]]-mean_2)
    two += (cora_graph.nodes('label')[edge[0]]-mean_1)*(cora_graph.nodes('label')[edge[0]]-mean_1)
    three += (cora_graph.nodes('label')[edge[1]]-mean_2)*(cora_graph.nodes('label')[edge[1]]-mean_2)

d = math.sqrt(two*three)
print(one/d)

In [11]:

attribute_assort = []
label_assort = []
degree_assort = []
density = []
clustering_coef  = []
avg_degree=[]    
if True:
        gr = chameleon_graph
        
        try:
            i_d=(data_ch.is_directed())
            isdirected=(i_d)
        except:
            print('no info')
        
        try:
            a_s = nx.attribute_assortativity_coefficient(gr,'feature')
            attribute_assort.append(a_s)
        except:
            attribute_assort.append(None)
        
            
        density.append(nx.density(gr))
             #ассортативность
        degree_assort.append(nx.degree_assortativity_coefficient(gr))
        clustering_coef.append(np.mean(list(nx.clustering(gr).values())))
        avg_degree.append(np.mean(np.array(list(dict(gr.degree).values()))))
        
        

In [11]:
number_of_nodes = []
clustering_coef=[]
density=[]
degree_assort=[]
number_of_features=[]
number_of_labels = []
attribute_assort = []
isdirected=[]
avg_degree = []
avg_path = []
 
    
for i,gr in enumerate(graphs):
        print(i)
        try:
            n_f=datasets[i].num_features
            number_of_features.append(n_f)
        except:
            if (datasets_names[i] == 'CoCit') or (datasets_names[i]=='BlogCatalog') or (datasets_names[i]=='Wikipedia'):
                number_of_features.append(datasets[i].shape(1))
            else:
                number_of_features.append(0)
        
        try:
            i_d=(datasets[i].is_directed())
            isdirected.append(i_d)
        except:
            if (datasets_names[i] == 'CoCit') or (datasets_names[i]=='BlogCatalog') or (datasets_names[i]=='Wikipedia'):
                isdirected.append(False)
            elif (datasets_names[i] == 'Arxiv') or (datasets_names[i]=='Epinion'):
                isdirected.append(True)
        
        try:
            n_l = len(collections.Counter(datasets[i].y.tolist()).keys())
            number_of_labels.append(n_l)
        except:
            if (datasets_names[i] == 'CoCit'):
                number_of_labels.append(len(collections.Counter(datasets[i].y).keys()))
            elif (datasets_names[i] == 'Arxiv') or (datasets_names[i]=='Epinion') or (datasets_names[i]=='BlogCatalog') or (datasets_names[i]=='Wikipedia'):
                number_of_labels.append(None)
        
        try:
            a_s = nx.attribute_assortativity_coefficient(gr,'feature')
            attribute_assort.append(a_s)
        except:
            attribute_assort.append(None)
        
            
        number_of_nodes.append(gr.number_of_nodes())
            #плотность
        density.append(nx.density(gr))
             #ассортативность
        degree_assort.append(nx.degree_assortativity_coefficient(gr))
        clustering_coef.append(np.mean(list(nx.clustering(gr).values())))
        avg_degree.append(np.mean(np.array(list(dict(graphs[1].degree).values()))))
        
        

0
1
2
3
4


In [15]:
datasets =[data_Cora, data_Citeseer, data_Pubmed, data_Photo, data_Computers]
datasets_names = ['Cora','Citeseer','Pubmed','Photo','Computers']
gr=graphs[3:]

In [22]:
for g in gr:
    ep=0
    n=0
    for component in (nx.connected_components(g)):
        ep+=nx.average_shortest_path_length(g.subgraph(component))
        n+=1
        print(ep)
    print('for gr, ', (ep/(n)))

KeyboardInterrupt: 

In [ ]:
try:
            ep = nx.average_shortest_path_length(gr)
            avg_path.append(ep)
        except:
            ep = 0
            n = 0
            for component in (nx.connected_components(gr)):
                ep+=nx.average_shortest_path_length(gr.subgraph(component))
                n+=1
            avg_path.append(ep/(n))

In [12]:
avg_path = [1.170483515230952, 1.0880632688377956, 6.336870027999047, 0.17807105484562516, 0.12205409485326828]#, 5.427693716383878]

In [13]:
df = pd.DataFrame({'Name of dataset' : datasets_names,
                                'Number of nodes (NN)' :number_of_nodes,'Number of features (NF)' :number_of_features,'Number of lables (NL)':number_of_labels , 'Directed? ' :isdirected,
                                'Clustering coefficient (CC)': clustering_coef,'Density(D)':density, 'Degree assortativity coefficient(DAC)':degree_assort, 'Attribute assortativity coefficient(AAC)': attribute_assort, 'Average Degree (AD)': avg_degree, 'Average shortest path': avg_path })
df.sort_values(by=['Attribute assortativity coefficient(AAC)']) #добавить атрибуты, направленность

,Name of dataset,Number of nodes (NN),Number of features (NF),Number of lables (NL),Directed?,Clustering coefficient (CC),Density(D),Degree assortativity coefficient(DAC),Attribute assortativity coefficient(AAC),Average Degree (AD),Average shortest path
0,Cora,2708,1433,7,False,0.240673,0.001440,-0.065871,-0.000850,3.898080,1.170484
1,Citeseer,3327,3703,6,False,0.141471,0.000823,0.048378,-0.000547,2.736399,1.088063
3,Photo,7650,745,8,False,0.403979,0.004070,-0.044943,-0.000427,31.132288,0.178071
4,Computers,13752,767,10,False,0.344126,0.002600,-0.056488,-0.000332,35.756399,0.122054
2,Pubmed,19717,500,3,False,0.060175,0.000228,-0.043640,-0.000189,4.496019,6.336870


In [ ]:
import pickle 
with open('analytics.pickle','wb') as f:
    pickle.dump(df,f)

In [ ]:
df.sort_values(by=['Pearson degree assortativity coefficient (PDAC)'])

In [ ]:
df.sort_values(by=['Degree assortativity coefficient(DAC)'])

In [ ]:
for dataset in datasets:
    print(datasets[i].x.shape)